# Pequeña charla sobre el desempeño de Python
                                                        Sergio Burdisso
                                            (sergio.burdisso@gmail.com)

## Todo tiene un costo...

El uso de Python dentro de muchas comunidades científicas está creciendo continuamente. Esto se debe en gran medida a que Python es un lenguaje **extremadamente flexible e "indulgente"**, esta flexibilidad lleva a un **uso eficiente del tiempo de desarollo** de programas. El hecho de que use tipos dinámicos (e.d. es débilmente tipado) hace que Python sea **más fácil de usar que otros lenguajes como C**.

Sin embargo, como se verá:
* Todo esta flexibilidad muchas veces nos lleva a **escribir código** que, una vez en ejecución, no es tan **eficiente**.
* La **flexibilidad de Python viene a costa de** una **pérdida en el desempeño**.

Trataremos el primer ítem en la siguiente sección, en el que se darán algunos concejos a tener en cuenta al momento de escribir programas en Python. Finalmente en la sección titulada _"¿Por qué Python es lento?"_ se tratará el segundo ítem, en el que intentaremos comprender por qué se dice que Python "es lento" para así ver qué medidas se pueden tomar para mitigar el problema.

## A tener en cuenta...

A continuación se daran una serie de concejos a tener en cuenta cuando se codifique en Python, muchos de ellos son genéricos y probablemente se puedan aplicar a otros lenguajes interpretados (tales como JavaScript) pero muchos otros son propios de Python.

### 0) Cuando sea necesario medir los tiempos de ejecución

Para medir el tiempo de ejecución se pueden usar, entre otras, las siguientes opciones:

* User módulo [*cProfile*](https://docs.python.org/2/library/profile.html) para ver el tiempo de ejecución en general e individual a cada función, así como también la cantidad de veces que cada función se llamó.

In [18]:
from cProfile import run

def uno():
    w = ""
    for _ in xrange(1000000):
        w+= "hola"
    return w

def dos():
    return "".join(["hola" for _ in xrange(1000000)])

run("uno()") #tottime 0.113 sec
run("dos()") #tottime 0.044 sec => (1 - 0.044/0.113 = 0.61) poco más de dos veces más rápido! (61%)

         3 function calls in 0.163 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.163    0.163    0.163    0.163 <ipython-input-18-85fee31f9e1d>:3(uno)
        1    0.000    0.000    0.163    0.163 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         4 function calls in 0.065 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.045    0.045    0.065    0.065 <ipython-input-18-85fee31f9e1d>:9(dos)
        1    0.000    0.000    0.065    0.065 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.019    0.019    0.019    0.019 {method 'join' of 'str' objects}




* Usar módulo [timeit](https://docs.python.org/2/library/timeit.html):

In [9]:
from timeit import timeit
timeit('"".join(["hola" for _ in xrange(1000)])', number=10000)

0.6078629493713379

* En Linux usar el comando _time_:
<pre>
time python script.py
time ./script.py
</pre>

### 1) Evitar operadores _._ y _[ ]_ tanto como se pueda

A diferencia de lenguajes compilados en los que los desplazamientos ya están calculados, en lenguajes como Python los operadores . y [] son costosos! se tiene que localizar el objeto, consultar su tipo y obtener (o calcular) el desplazamiento de el elemento a seleccionar dentro de ese objeto.

**Solución:** usar variables a modo de caché para retener el valor para luego operar.

In [17]:
class Dummy:
    campo = {
        "key0": ["value"]*1000000,
        "key1": ["value"]
    }


def uno():
    d = Dummy()
    for i in xrange(len(d.campo["key0"])):
        d.campo["key0"][i] = "new value"
        d.campo["key1"][0] = i
#VS.

def dos():
    d = Dummy()
    key0s = d.campo["key0"]
    key1 = d.campo["key1"]
    for i in xrange(len(key0s)):
        key0s[i] = "new value"
        key1[0] = i

run("uno()") #tottime 0.260 sec
run("dos()") #tottime 0.090 sec => más del doble rápido! (65.3%)

         4 function calls in 0.260 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.260    0.260    0.260    0.260 <ipython-input-17-e593007b18d8>:9(uno)
        1    0.000    0.000    0.260    0.260 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         4 function calls in 0.090 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.090    0.090    0.090    0.090 <ipython-input-17-e593007b18d8>:16(dos)
        1    0.000    0.000    0.090    0.090 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### 2) Usar variables locales

En Python el acceso a las variables no locales (e.g. globales) es más costo que al de las variables locales.

**Solución:** usar variables locales, nuevamente a modo de caché, para mantener el valor de la variable no local.

In [20]:
VALOR0 = 10
VALOR1 = 20

def uno():
    res = 0
    for i in xrange(1000000):
        res += VALOR0 + VALOR1

#VS.

def dos():
    res = 0
    valor0 = VALOR0
    valor1= VALOR1
    for i in xrange(1000000):
        res += valor0 + valor1

run("uno()") #tottime 0.116 sec
run("dos()") #tottime 0.061 sec => casi el doble rápido! (47.4%)

         3 function calls in 0.116 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.116    0.116    0.116    0.116 <ipython-input-20-575760f51201>:6(uno)
        1    0.000    0.000    0.116    0.116 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         3 function calls in 0.061 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.061    0.061    0.061    0.061 <ipython-input-20-575760f51201>:13(dos)
        1    0.000    0.000    0.061    0.061 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### 3) Evitar recomputar valores constante

Si un valor se mantendrá constante sobre alguna parte del cómputo, sobre todo en los loops, usar variable local a modo de caché para retener ese valor. 

In [24]:
def uno():
    errorNbr = 13
    msg = ""
    for i in xrange(1000000):
        msg = "Error %d: value was %d"%(errorNbr, i)

#VS.

def dos():
    errorNbr = 13
    MSG =("Error %d: value was " % errorNbr) + "%d"
    msg = ""
    for i in xrange(1000000):
        msg = MSG % i

run("uno()") #tottime 1.322 sec
run("dos()") #tottime 0.702 sec => casi el doble rápido! (46.8%)

         3 function calls in 1.322 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.322    1.322    1.322    1.322 <ipython-input-24-e3897ce436a2>:3(uno)
        1    0.000    0.000    1.322    1.322 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         3 function calls in 0.702 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.702    0.702    0.702    0.702 <ipython-input-24-e3897ce436a2>:11(dos)
        1    0.000    0.000    0.702    0.702 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### 4) Evitar usar loops clásicos

Siempre que sea posible, evitar de el uso "clásico" de los iteradores como for y while y usar las variantes propias de Python como los _generadores_ y las funciones [_map_](https://docs.python.org/2/library/functions.html#map) y [_reduce_](https://docs.python.org/2/library/functions.html#reduce) ya que están construidas al rededor de C:

In [23]:
# Supongamos que queremos pasar todas las palabras de
# la siguiente lista a mayúscula
words = ["imasl"] * 1000000

#loop clásico
def uno():
    WORDS = []
    for i in xrange(len(words)):
        WORDS.append(words[i].upper())
    return WORDS

#usando generador
def dos():
    return [w.upper() for w in words]

#usando map con funciones predefinidas
def tres():
    return map(str.upper, words)

#usando map con funciones lambda
def cuatro():
    return map(lambda w: w.upper(), words)

run("uno()") # cumtime 0.655
run("dos()") # cumtime 0.315 => mejor que uno, el doble de rápido (52%)
run("tres()")# cumtime 0.160 => MEJOR, cuatro veces más rápido que uno!! (75%)
run("cuatro()")# cumtime 0.665 => noy hay una diferencia sifnificativa con respecto a uno

#NOTA: 
#    * A diferencia de la primera opción (uno) en las otras tres la paralelización del loop está "explicita"
#      ya que en su declaración se hace evidente que cada iteración es independiente de la otra.
#    * map es una excelente opción siempre que podamos utilizarlo sin el uso de funciones lambda y preferentemente
#      con funciones incorporadas (built-in) o construidas al rededor de C.

         2000004 function calls in 0.672 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.455    0.455    0.655    0.655 <ipython-input-23-f8eb0f6f7670>:6(uno)
        1    0.016    0.016    0.672    0.672 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {len}
  1000000    0.069    0.000    0.069    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  1000000    0.132    0.000    0.132    0.000 {method 'upper' of 'str' objects}


         1000003 function calls in 0.331 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.190    0.190    0.315    0.315 <ipython-input-23-f8eb0f6f7670>:13(dos)
        1    0.017    0.017    0.331    0.331 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Prof

### 5) Usar componentes _lazy_

Siempre que sea posible usar componentes lazy, esto es, los elementos se crearán al momentos de necesitarlos, para evitar que se ocupe mucho espacio en memoria, sobre todo cuando se trata con una gran cantidad de datos.
* Lazy imports
* Lazy rangos
* Lazy listas (generadores)

In [37]:
#IMPORTS
def uno():
    import re
    return re.search
#VS
import re
def dos():
    return re.search
#VS
re = None
def tres():
    global re
    if re is None:
        import re
    return re.search

run("for _ in xrange(100000): uno()") # cumtime 0.101
run("import re\nfor _ in xrange(100000): dos()") # cumtime 0.013 => mucho mejor, 87% más rápido
run("for _ in xrange(100000): tres()")# cumtime 0.027 => MEJOR, casi igual a dos ¡pero sin desperdiciar espacio!

         100002 function calls in 0.142 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.101    0.000    0.101    0.000 <ipython-input-37-2a24ddbcab5d>:1(uno)
        1    0.041    0.041    0.142    0.142 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         100002 function calls in 0.037 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.013    0.000    0.013    0.000 <ipython-input-37-2a24ddbcab5d>:6(dos)
        1    0.024    0.024    0.037    0.037 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         100002 function calls in 0.063 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.027    0.000    0.027    0.000 <ip

In [2]:
#RANGOS
def uno():
    #range crea los 50000000 elementos
    for _ in range(50000000): pass

def dos():
    #xrange crea solamete el elemento
    #cuando se necesita, primero 0, luego 1, 2, 3... etc.
    for _ in xrange(50000000): pass

#ejecutar y ver uso de memoria

In [3]:
#LISTAS
def uno():
    #[ .. for in ..] crea los 50000000 elementos
    return [n**2 for n in xrange(50000000)]

def dos():
    #( .. for in ..) crea solamete el elemento
    #cuando se necesita, primero 0, luego 1, 4, 9... etc.
    return (n**2 for n in xrange(50000000))

#ejecutar y ver uso de memoria

### 6) Algunos más...

* Mover loops dentro de funciones si el cuerpo del loop es una función ¡las **llamadas a funciones/métodos** también son **costosas**! :)
<pre>
    for ...
        func()
    vs.
    func():
        for ...
</pre>
* Una función que pregunta por condición para ejecutar, reemplazar condición por try except (o usar decorador). Sino va a preguntar todas las veces por condición cuando en realidad es "la excepcion y no la regla".
<pre>
    inc():
        if e is None:
            # this chunk is executed only once
            e = 0
        else:
            e += 1
    vs.
    inc():
        try:
            e += 1
        except:
            e = 0
</pre>
* Etc. :D

## ¿Por qué Python es lento?

Todos hemos escuchado alguna vez decir: *"Python es lento"*

Esto no quiere decir necesariamente que sea lento en un sentido literal, sino más bien se debe a que, a diferencia de lenguajes como C, internamente el interprete de Python debe hacer un trabajo extra para brindar el soporte necesario para la ejecución de los programas. Puntualmente se debe a los siguientes factores:

### 1) Python usa "tipos dinámicos" en lugar de "tipos estáticos"

A diferencia de lenguajes como C, cuando un programa Python ejecuta, el interprete no sabe el tipo de cada variable definida, por lo que debe mantener suficiente información durante la ejecución del mismo como para poder identificar el tipo de cada variable y operar sobre las misma. Por lo que durante ejecución, por ejemplo, en memoria una variable cuyo valor sea 4 (tipo entero) en C y Python tendrán la siguiente forma:

![img](imgs/cint_pyint_mem.png "Estructura de entero en C y en Python")

*(Pequeño hack! a modo de prueba de concepto, cambiemos el valor del objeto "4" a 5, de forma tal que para Python 2+2 sea igual a 5:)*

*Analizando el [código fuente](https://hg.python.org/cpython/file/2.7/) en C de la implementación de "Python" podemos comprender la estructura real de estos objetos. Para el caso de objetos con valores entero hay que observar los archivos [longintrepr.h](https://hg.python.org/cpython/file/2.7/Include/longintrepr.h/#l90) y [object.h](https://hg.python.org/cpython/file/2.7/Include/object.h#l96). Una vez hecho esto podemos escribir un programa python que primero averigüe el desplazamiento del campo **ob_digit** y luego cambie su contenido de 4 a 5, esto es:*


```python
#**NOTA** Ejecutarlo solamente desde la terminal.
#         NO ejecutar desde ninguna Notebook porque probablemente rompa el kernel de Jupyter.
import ctypes

address_of_4 = id(4)

# adivinar el desplazaminto de ob_digit dentro de los objetos de tipo entero
for offset in xrange(16):
    fields = [("dummy", ctypes.c_char)]*offset + [("ob_digit", ctypes.c_long)]
    class IntStruct(ctypes.Structure):
        _fields_ = fields
        def __repr__(self):
            return ("IntStruct(ob_digit={self.ob_digit})").format(self=self)
    if IntStruct.from_address(address_of_4).ob_digit == 4:
        break

# reemplacemos el valor de ob_digit por 5
IntStruct.from_address(address_of_4).ob_digit = 5

print 2+2
```

Como en Python cada valor es un objeto "complejo" si se tuviera, por ejemplo, el siguiente código escrito en C y Python, en ejecución se realizarían las siguientes operaciones:

|  	| C 	| Python 	|
|--------------------------	|---------------------------------------------------------------------------------------	|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| **Código** 	| int x, y z;<br>x = 2;<br>y = 2;<br>z = x + y; 	|  x = 2<br>y = 2<br>z = x + y 	|
| **Operaciones<br>(en ejecución)** 	| 1) asignar 2 a x<br>2) asignar 2 a y<br>3) suma binaria de x con y<br>4) asignar resultado a z 	| 1.a) asignar Entero a x->PyObject_HEAD->type<br>1.b) asignar 2 a x->digit<br>2.a) asignar Entero a y->PyObject_HEAD->type<br>2.b) asignar 2 a y->digit<br>3.a) buscar type en x->PyObject_HEAD<br>3.b) x es un entero; el valor es x->digit<br>3.c) buscar type en y->PyObject_HEAD<br>3.d) y es un entero; el valor es y->digit<br>3.e) suma binaria de x->digit con y->digit<br>4.a) asignar Entero a z->PyObject_HEAD->type<br>4.a) asignar resultado de suma a z->digit 	|

*Nota: Estas operaciones abstractas son solamente ilustrativas, en la práctica se crea un objeto por cada valor en Python, y las variables apuntan/referencian a dichos objetos. Para obtener cuantas variables apuntan a dicho objeto-valor se puede usar la función [**getrefcount**](https://docs.python.org/2/library/sys.html#sys.getrefcount) del módulo [**sys**](https://docs.python.org/2/library/sys.html):*

In [12]:
from sys import getrefcount

print "Referencias al valor(objeto) 2:   ", getrefcount(2)
print "Referencias al valor(objeto) 1024:", getrefcount(1024)

Referencias al valor(objeto) 2:    902
Referencias al valor(objeto) 1024: 3


El uso de **tipos dinámicos** lleva a que existan **más pasos involucrados en cualquier operación** y que se tengan que usar estructuras para mantener información importante para cada objeto, como ser su tipo.

### 2) Python es interpretado en lugar de compilado

Arriba se mostró una diferencia entre código interpretado y compilado. Un compilador "inteligente" puede hacer uso de los tipos y las operaciones para producir código en lenguaje máquina eficiente y posiblemente optimizando (eliminando código repetido o innecesario).

### 3) Python puede llevar a un uso ineficiente de la memoria

El esquema valor-objeto de Python puede llevar a un uso ineficiente de la memoria (tanto en accesos como en espacio), por ejemplo, supongamos que se tiene un conjunto de valores enteros y se necesita realizar una operación sobre todos ellos, uno después de otro. Para esto se podría usar el objeto *List* mientras que en C se utilizaría un arreglo de valores enteros contiguos en memoria, lo que se ilustra en la siguiente figura:
![img](imgs/array_vs_list.png "Estructura de arreglo en C y List en Python")

Es fácil observar que cualquiera operación en secuencia sobre estos elementos producirá un uso más ineficiente de la memoria tanto en espacio como en accesos, por lo que algunas soluciones podrían ser:
* [Cython](http://cython.org/) -- restringir algunas características de Python y expandir su sintaxis para poder compilarlo (e.g. declarar el tipo de las variables)
* [Numba](http://numba.pydata.org/) -- con el uso de algunos @decoradores podremos hacer que nuestras funciones sean compiladas en tiempo de ejecución. Esto es, al momento que se llama una de estas funciones, Numba observará el tipo de cada parámetro y luego procederá a realizar la compilación de la misma. Esto se conoce como "Compilación Justo a Tiempo" o en Inglés Just In Time(JIT) Compilation. Va a depender de las características de cada función (qué tan "compilable" sea) la calidad de la JIT-compilación y por ende la mejora de rendimiento que obtendremos.
* Usar objetos Python construidos al rededor de C como los provistos por algunos frameworks o bibliotecas especiales, como ser [NumPy](http://www.numpy.org/). Por ejemplo, la estructura de un objeto _arreglo Numpy_ sería la siguiente:

![img](imgs/numpy_array.png "Estructura de arreglo Numpy")

* Crear módulos propios en C: _**¿por qué no construir nuestros propios módulos Python en C?** creemos el modulo IMASL, el cual contendrá sólo una función llamada "pydata" que al ser llamada con una string y, opcionalmente un número, nos devolverá algo misterioso :D_